### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### The idea is to create one notebook for each prefix. You only need to define the prefix in the follow cell and the notebook will do the rest.

***
# **1. Imports**

In [1]:
import sys
import pandas as pd

import get_properties_functions_for_WI as gp

import importlib

/home/intel00000/stahl_data_science/DFT Calculations/gp_start_folder_empty/get_properties_functions_for_WI.py:1549: SyntaxWarning: invalid escape sequence '\.'
  orbital_section = re.search("[0-9]+\.[A-Z\*(0-9 ]+\)",filecont[j]).group(0) #type of MO


***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **!!!User input required, Change the prefix to the one you want to handle.** </p>

In [2]:
prefix = "pyrd"

file_name = prefix + "_atom_map.xlsx"

atom_map_df = pd.read_excel(
    file_name, "Sheet1", index_col=0, header=0, engine="openpyxl"
)

display(atom_map_df.head())

df = atom_map_df  # df is what properties will be appended to, this creates a copy so that you have the original preserved

,log_name,C3,C4,N1,C5,C6,C7,C1,C2
0,pyrd10_conf-1,C6,C5,N4,C3,C12,C7,C2,C3
1,pyrd10_conf-2,C6,C5,N4,C3,C12,C7,C2,C3
2,pyrd11_conf-1,C4,C3,N12,C11,C10,C5,C2,C3
3,pyrd12_conf-1,C3,C4,N5,C6,C7,C12,C2,C3
4,pyrd12_conf-2,C3,C4,N5,C6,C7,C12,C2,C3


In [3]:
# testing code for subprocess and goodvibes

importlib.reload(gp)
import subprocess

log_file = "pyrd1_conf-1.log"

# Construct command-line arguments for the new version of goodvibes
cmd_args = [
    sys.executable, "-m",
    "goodvibes", 
    log_file,
    "--spc", "link",
    "-t", str(298.15)
]

# Run the goodvibes command and capture the output
result = subprocess.run(cmd_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# print the output
print(result.stdout)

   GoodVibes v3.2 2024/10/17 17:30:57
   Citation: Luchini, G.; Alegre-Requena, J. V.; Funes-Ardoiz, I.; Paton, R. S. F1000Research, 2020, 9, 291.
   GoodVibes version 3.2 DOI: 10.12688/f1000research.22758.1

o  Requested: --spc link -t 298.15 

   Temperature = 298.15 Kelvin   Pressure = 1 atm
   All energetic values below shown in Hartree unless otherwise specified.

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Entropic quasi-harmonic treatment: frequency cut-off value of 100.0 wavenumbers will be applied.
   QS = Grimme: Using a mixture of RRHO and Free-rotor vibrational entropies.
   REF: Grimme, S. Chem. Eur. J. 2012, 18, 9955-9964

   Combining final single point energy with thermal corrections.

   Structure                                       E_SPC             E        ZPE         H_SPC        T.S     T.qh-S      G(T)_SPC   qh-G(T)_SPC
   *********************************************************************************************************

# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [4]:
# this box has functions to choose from
df = atom_map_df

# ---------------GoodVibes Engergies---------------
# uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
# calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
# inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

# ---------------Frontier Orbitals-----------------
# E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

# ---------------Polarizability--------------------
# Exact polarizability
df = gp.get_polarizability(df)

# ---------------Dipole----------------------------
# Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

# ---------------Volume----------------------------
# Molar volume
# requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

# ---------------SASA------------------------------
# Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

# ---------------NBO-------------------------------
# natural charge from NBO
# requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "C2", "N1"]
df = gp.get_nbo(df, nbo_list)

# ---------------NMR-------------------------------
# isotropic NMR shift
# requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C2", "N1"]
df = gp.get_nmr(df, nmr_list)

# ---------------Distance--------------------------
# distance between 2 atoms
dist_list_of_lists = [["C1", "C2"]]
df = gp.get_distance(df, dist_list_of_lists)

# ---------------Angle-----------------------------
# angle between 3 atoms
# angle_list_of_lists = [["C5", "N1", "C1"]]
# df = gp.get_angles(df, angle_list_of_lists)

# ---------------Dihedral--------------------------
# dihedral angle between 4 atoms
# dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
# df = gp.get_dihedral(df, dihedral_list_of_lists)

# ---------------Vbur Scan-------------------------
# uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
# inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
# if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C2"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

# ---------------Sterimol morfeus------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values
# NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["C1", "C2"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

# ---------------Buried Sterimol-------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
# atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
# for more information: https://kjelljorner.github.io/morfeus/sterimol.html
# inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["C1", "C2"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

# ---------------Sterimol DBSTEP-------------------
# uses DBSTEP to calculate Sterimol L, B1, and B5 values
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
# sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

# ---------------Sterimol2Vec----------------------
# uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# inputs: dataframe, list of atom pairs, end_radius, and step_size
# sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

# ---------------Pyramidalization------------------
# uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
# collects values based on two definitions of pyramidalization
# details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C2"]
df = gp.get_pyramidalization(df, pyr_list)

# ---------------Plane Angle-----------------------
# !plane angle between 2 planes (each defined by 6 atoms)
# planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
# df = gp.get_planeangle(df, planeangle_list_of_lists)

# --------------LP energy - custom from first cell---------------
lp_list = ["N1"]
df = gp.get_one_lp_energy(df, lp_list)

# ---------------Time----------------------------------
# returns the total CPU time and total Wall time (not per subjob) because we are pioneers
# if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
# df = gp.get_time(df)

# ---------------ChelpG----------------------------
# ChelpG ESP charge
# requires the Gaussian keyword = "pop=chelpg" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_chelpg(df, a_list)

# ---------------Hirshfeld-------------------------
# Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
# requires the Gaussian keyword = "pop=hirshfeld" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_hirshfeld(df, a_list)

pd.options.display.max_columns = None
display(df)

Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2', 'N1']
NMR function has completed for ['C1', 'C2', 'N1']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 4
Morfeus Sterimol function has completed for [['C1', 'C2']]
Morfeus Buried Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1', 'C2']
{'NBO_LP_occupancy_N1': '1.91671', 'NBO_LP_energy_N1': '-0.37134'}
{'NBO_LP_occupancy_N1': '1.91416', 'NBO_LP_energy_N1': '-0.37054'}
{'NBO_LP_occupancy_N1': '1.91590', 'NBO_LP_energy_N1': '-0.37504'}
{'NBO_LP_occupancy_N1': '1.91596', 'NBO_LP_energy_N1': '-0.37555'}
{'NBO_LP_occupancy_N1': '1.91708', 'NBO_LP_energy_N1': '-0.37448'}
{'NBO_LP_occupancy_N1': '1.91789', 'NBO_LP_energy_N1': '-0.37327'}
{'N

/home/intel00000/stahl_data_science/.venv/lib/python3.12/site-packages/morfeus/pyramidalization.py:165: RuntimeWarning: invalid value encountered in sqrt
  P_angle = np.sqrt(360 - sum(thetas))


,log_name,C3,C4,N1,C5,C6,C7,C1,C2,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_N1,NMR_shift_C1,NMR_shift_C2,NMR_shift_N1,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,Buried_Sterimol_L_C1_C2_5.0(Å),Buried_Sterimol_B1_C1_C2_5.0(Å),Buried_Sterimol_B5_C1_C2_5.0(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,NBO_LP_occupancy_N1,NBO_LP_energy_N1
0,pyrd10_conf-1,C6,C5,N4,C3,C12,C7,C2,C3,-480.519907,0.192685,-480.316691,0.045527,0.045073,-480.362218,-480.361764,298.15,-0.27734,-0.02776,-0.152550,0.24958,0.04662,132.6560,84.2449,2.1475,1331.512,344.422329,510.979600,0.897419,-0.43973,0.27254,-0.43683,151.4703,-6.4711,-139.0471,1.51130,97.307593,96.313275,84.174372,88.507618,68.804939,78.545888,55.588427,67.206136,43.308531,55.398098,7.038742,1.728865,5.725857,6.847921,1.728865,5.725857,5.910457,0.799290,0.188998,2.838579e-02,1.91671,-0.37134
1,pyrd10_conf-2,C6,C5,N4,C3,C12,C7,C2,C3,-480.520348,0.192433,-480.317344,0.045613,0.045152,-480.362957,-480.362496,298.15,-0.27786,-0.02727,-0.152565,0.25059,0.04644,133.0260,91.3154,2.1208,1324.025,346.782431,512.468806,0.893042,-0.44193,0.27382,-0.44353,154.6454,-2.1970,-133.6838,1.51572,97.933884,95.538481,84.693645,87.312801,69.141091,77.849374,55.749744,66.754216,43.370989,55.123828,6.993851,1.974626,5.734566,6.860939,1.974626,5.734566,6.130172,0.821118,0.000000,5.110653e-11,1.91416,-0.37054
2,pyrd11_conf-1,C4,C3,N12,C11,C10,C5,C2,C3,-480.521016,0.192386,-480.318012,0.046041,0.045268,-480.364052,-480.363280,298.15,-0.27694,-0.02718,-0.152060,0.24976,0.04629,135.9940,104.5980,1.9903,1593.997,353.302391,519.415045,0.884465,-0.42344,0.19758,-0.43324,149.3872,0.4796,-143.4645,1.50789,97.007361,96.245480,82.437492,88.038807,65.944391,76.650961,52.233391,63.516236,39.780662,50.630004,9.049785,1.881765,4.340607,7.782319,1.881765,4.340607,5.892928,0.797979,0.098666,1.480590e-02,1.91590,-0.37504
3,pyrd12_conf-1,C3,C4,N5,C6,C7,C12,C2,C3,-480.517722,0.192841,-480.314382,0.045334,0.045037,-480.359715,-480.359419,298.15,-0.27747,-0.02883,-0.153150,0.24864,0.04717,132.4390,84.9530,2.6646,1211.966,342.122157,509.133637,0.901275,-0.42167,-0.02736,-0.41154,158.2171,30.6890,-145.4243,1.50907,97.501291,97.356018,84.721318,89.975908,69.806876,80.449195,57.007082,68.960236,44.615480,56.165430,7.069170,1.771042,5.717505,6.845429,1.771042,5.717505,5.916355,0.799873,0.162777,2.448454e-02,1.91596,-0.37555
4,pyrd12_conf-2,C3,C4,N5,C6,C7,C12,C2,C3,-480.517209,0.192885,-480.313861,0.045291,0.044909,-480.359151,-480.358770,298.15,-0.27814,-0.02848,-0.153310,0.24966,0.04707,132.7650,92.0522,2.7010,1580.972,343.332189,509.577023,0.898620,-0.42214,-0.02687,-0.41205,160.5298,34.1815,-143.3000,1.51326,98.050103,96.729726,84.924795,88.636216,69.747281,79.334587,56.937780,68.109392,44.582894,55.672367,6.993240,1.975913,5.733793,6.864294,1.975913,5.733793,6.137171,0.821798,0.000000,4.815293e-11,1.91708,-0.37448
5,pyrd13_conf-1,C5,C4,N3,C2,C7,C6,C1,C2,-287.578875,0.116330,-287.455541,0.037368,0.036781,-287.492909,-287.492322,298.15,-0.30859,0.00211,-0.153240,0.31070,0.03779,73.0994,43.1907,1.8769,828.737,258.341581,349.336777,0.928509,-0.63711,0.21058,-0.43001,159.0468,-1.5126,-151.2781,1.50650,93.084969,95.716038,71.205561,85.758237,51.545739,72.466292,39.510925,57.646524,31.191810,43.329480,6.831659,1.886043,3.341965,6.831659,1.886043,3.341965,5.965683,0.805395,0.000021,3.229065e-06,1.91789,-0.37327
6,pyrd14_conf-1,C2,C7,N6,C5,C4,C3,C1,C2,-287.576157,0.116447,-287.452703,0.037376,0.036794,-287.490079,-

## 3.1 Save collected properties to Excel and pickle file

In [5]:
# save the pandas dataframe to a xlsx file
with pd.ExcelWriter(prefix + "_extracted_properties.xlsx") as writer:
    df.to_excel(writer)
# save the pandas dataframe to a pickle file
df.to_pickle(prefix + "_extracted_properties.pkl")

# **4. Post-processing**

In [6]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [7]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "pyrd"
suffix = "_"

# columns that provide atom mapping information are dropped, not need if these columns contain cells that cannot be convert to float
# (which is the case for C1, C2, C3, C4, C5, N1 but not pure numbers like 1, 2, 3, 4, 5, 6)
atom_columns_to_drop = ["C3", "C4", "C5", "N1", "C1", "C2"]

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [8]:
df = pd.read_excel(
    prefix + "_extracted_properties.xlsx",
    "Sheet1",
    index_col=0,
    header=0,
    engine="openpyxl",
)

display(df.head())

,log_name,C3,C4,N1,C5,C6,C7,C1,C2,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_N1,NMR_shift_C1,NMR_shift_C2,NMR_shift_N1,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,Buried_Sterimol_L_C1_C2_5.0(Å),Buried_Sterimol_B1_C1_C2_5.0(Å),Buried_Sterimol_B5_C1_C2_5.0(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,NBO_LP_occupancy_N1,NBO_LP_energy_N1
0,pyrd10_conf-1,C6,C5,N4,C3,C12,C7,C2,C3,-480.519907,0.192685,-480.316691,0.045527,0.045073,-480.362218,-480.361764,298.15,-0.27734,-0.02776,-0.152550,0.24958,0.04662,132.656,84.2449,2.1475,1331.512,344.422329,510.979600,0.897419,-0.43973,0.27254,-0.43683,151.4703,-6.4711,-139.0471,1.51130,97.307593,96.313275,84.174372,88.507618,68.804939,78.545888,55.588427,67.206136,43.308531,55.398098,7.038742,1.728865,5.725857,6.847921,1.728865,5.725857,5.910457,0.799290,0.188998,2.838579e-02,1.91671,-0.37134
1,pyrd10_conf-2,C6,C5,N4,C3,C12,C7,C2,C3,-480.520348,0.192433,-480.317344,0.045613,0.045152,-480.362957,-480.362496,298.15,-0.27786,-0.02727,-0.152565,0.25059,0.04644,133.026,91.3154,2.1208,1324.025,346.782431,512.468806,0.893042,-0.44193,0.27382,-0.44353,154.6454,-2.1970,-133.6838,1.51572,97.933884,95.538481,84.693645,87.312801,69.141091,77.849374,55.749744,66.754216,43.370989,55.123828,6.993851,1.974626,5.734566,6.860939,1.974626,5.734566,6.130172,0.821118,0.000000,5.110653e-11,1.91416,-0.37054
2,pyrd11_conf-1,C4,C3,N12,C11,C10,C5,C2,C3,-480.521016,0.192386,-480.318012,0.046041,0.045268,-480.364052,-480.363280,298.15,-0.27694,-0.02718,-0.152060,0.24976,0.04629,135.994,104.5980,1.9903,1593.997,353.302391,519.415045,0.884465,-0.42344,0.19758,-0.43324,149.3872,0.4796,-143.4645,1.50789,97.007361,96.245480,82.437492,88.038807,65.944391,76.650961,52.233391,63.516236,39.780662,50.630004,9.049785,1.881765,4.340607,7.782319,1.881765,4.340607,5.892928,0.797979,0.098666,1.480590e-02,1.91590,-0.37504
3,pyrd12_conf-1,C3,C4,N5,C6,C7,C12,C2,C3,-480.517722,0.192841,-480.314382,0.045334,0.045037,-480.359715,-480.359419,298.15,-0.27747,-0.02883,-0.153150,0.24864,0.04717,132.439,84.9530,2.6646,1211.966,342.122157,509.133637,0.901275,-0.42167,-0.02736,-0.41154,158.2171,30.6890,-145.4243,1.50907,97.501291,97.356018,84.721318,89.975908,69.806876,80.449195,57.007082,68.960236,44.615480,56.165430,7.069170,1.771042,5.717505,6.845429,1.771042,5.717505,5.916355,0.799873,0.162777,2.448454e-02,1.91596,-0.37555
4,pyrd12_conf-2,C3,C4,N5,C6,C7,C12,C2,C3,-480.517209,0.192885,-480.313861,0.045291,0.044909,-480.359151,-480.358770,298.15,-0.27814,-0.02848,-0.153310,0.24966,0.04707,132.765,92.0522,2.7010,1580.972,343.332189,509.577023,0.898620,-0.42214,-0.02687,-0.41205,160.5298,34.1815,-143.3000,1.51326,98.050103,96.729726,84.924795,88.636216,69.747281,79.334587,56.937780,68.109392,44.582894,55.672367,6.993240,1.975913,5.733793,6.864294,1.975913,5.733793,6.137171,0.821798,0.000000,4.815293e-11,1.91708,-0.37448


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [9]:
compound_list = []

for index, row in df.iterrows():
    log_file = row["log_name"]  # read file name from df
    prefix_and_compound = log_file.split(
        str(suffix)
    )  # splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(
        str(prefix)
    )  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(
    set(compound_list)
)  # removes duplicate stuctures that result from having conformers of each
compound_list.sort(
    key=lambda x: int(re.search(r"\d+", x).group())
)  # reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

# this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [10]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list:
    # defines the common start to all files using the input above
    substring = str(prefix) + str(compound) + str(suffix)

    # makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(
        drop=True
    )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

    # filter column that are characters, we will attempt to convert them to numbers, if fail, we will drop them
    for column in valuesdf:
        try:
            # exclude column "log_name"
            if column == "log_name":
                continue
            valuesdf[column] = pd.to_numeric(valuesdf[column])
        except:
            valuesdf = valuesdf.drop(columns=column)
            valuesdf = valuesdf.reset_index(
                drop=True
            )  # reset the index after dropping columns

    # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = [
        "G(Hartree)",
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]  # don't boltzman average columns containing these strings in the column label
    reg_avg_columns = [
        "CPU_time_total(hours)",
        "Wall_time_total(hours)",
    ]  # don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = [
        "E_spc (Hartree)",
        "H_spc(Hartree)",
        "T",
        "T*S",
        "T*qh_S",
        "ZPE(Hartree)",
        "qh_G(T)_spc(Hartree)",
        "G(T)_spc(Hartree)",
    ]
    gv_extra_columns.remove(str(energy_col_header))

    # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = (
        valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    )
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp(
        (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
    )  # R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row = []
    values_range_row = []
    values_exclude_columns = []

    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column)  # used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(
            phrase in column for phrase in gv_extra_columns
        ):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(
                valuesdf[column].mean()
            )  # intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(
                valuesdf[column]
            )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append(
                (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
            )
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            # this section generates the weighted std deviation (weighted by mole fraction)
            # formula: https://www.statology.org/weighted-standard-deviation-excel/

            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
            delta_values_sq = []

            # makes a list of the "deviation" for each conformer
            for index, row in valuesdf.iterrows():
                value = row[column]
                delta_value_sq = (value - boltz) ** 2
                delta_values_sq.append(delta_value_sq)

            # w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            # !swap the order here to avoid division by zero error
            if (
                len(w) == 1
            ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                wstdev = 0
            else:
                # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
                wstdev = np.sqrt(
                    (np.average(delta_values_sq, weights=w))
                    / (((len(w) - 1) / len(w)) * np.sum(w))
                )
            values_boltz_stdev_row.append(wstdev)

    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    valuesdf.loc[len(valuesdf)] = values_range_row

    # final output format is built here:
    explicit_order_front_columns = [
        "log_name",
        energy_col_header,
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]

    # reorders the dataframe using front columns defined above
    valuesdf = valuesdf[
        explicit_order_front_columns
        + [
            col
            for col in valuesdf.columns
            if col not in explicit_order_front_columns
            and col not in values_exclude_columns
        ]
    ]

    # determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    # copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
    new_row["log_name"] = "Lowest E Conformer"
    # check if there is empty/Nan values in the row
    if new_row.isnull().values.any():
        print("There are empty values in the row")
        # print the entire row to see where the empty values are
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
        # drop the columns with empty values
        new_row = new_row.dropna(axis=1, how="any")
        print("The empty values have been dropped")
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

    # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
    # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    # this template can be adjusted for min/max/etc.

    # find the index for the min or max column:
    ensemble_min_index = valuesdf[
        valuesdf["log_name"] == "Ensemble Minimum"
    ].index.tolist()
    # find the min or max value of the property (based on index above)
    # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_3.0Å"].tolist()
    vbur_min_index = valuesdf[valuesdf["%Vbur_C1_3.0Å"] == min_value[0]].index.tolist()
    # copy the row to a new_row with the name of the log changed to Property_min_conformer
    new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
    new_row["log_name"] = "%Vbur_C1_3.0Å_min_Conformer"
    # check if there is empty/Nan values in the row
    if new_row.isnull().values.any():
        print("There are empty values in the row")
        # print the entire row to see where the empty values are
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
        # drop the columns with empty values
        new_row = new_row.dropna(axis=1, how="any")
        print("The empty values have been dropped")
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)
    # --------------------------------------------------------------------------------------------------------------------

    # appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])

    # drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop=True)

    # drop the columns created to determine the mole fraction and some that
    valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns=gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns=reg_avg_columns)
    except:
        pass

    # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
    compound_name = prefix + str(compound)
    # --------------------------------------------------------------------------------------------------------------------

    properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

    # builds a dataframe (for each compound) by adding summary properties as new columns
    for column in valuesdf:
        # the indexes need to match the values dataframe - display it to double check if you need to make changes
        # (uncomment the display(valuesdf) in row 124 of this cell)

        # create a list of headers for the properties_df
        # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
        headers = [
            f"{column}_Boltz",
            f"{column}_Boltz_stdev",
            f"{column}_min",
            f"{column}_max",
            f"{column}_range",
            f"{column}_low_E",
            f"{column}_Vbur_min",
        ]
        # Extract values for the current column from valuesdf and create a DataFrame
        row_dataframe = pd.DataFrame([valuesdf[column].values], columns=headers)
        # Display the DataFrame for verification
        # display(row_dataframe)
        # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
        properties_df = pd.concat([properties_df, row_dataframe], axis=1)

    # concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

# Reset the index of the master DataFrames
all_df_master = all_df_master.reset_index(drop=True)
properties_df_master = properties_df_master.reset_index(drop=True)

There are empty values in the row
+---+--------------------+-------------------+-------------+--------------+------------+---------------+-----------------+--------------+----------------+----------+----------+----------------------+--------+--------+----------+-----------+---------+---------+------------------+--------------------+---------------+--------------------------+-----------------------+-------------------+--------------------+---------------+---------------+---------------+--------------+--------------+--------------+-------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------------------+------------------------------+------------------------------+--------------------------------+---------------------------------+---------------------------------+--------------------------------+---------------------

# 5. Export the data

In [11]:
# Print in tabulated version
print(tabulate(properties_df_master, headers="keys", tablefmt="pretty"))
print(tabulate(all_df_master, headers="keys", tablefmt="pretty"))

# grep the first & last Compound_Name
first_compound = str(properties_df_master["Compound_Name"].iloc[0])
last_compound = str(properties_df_master["Compound_Name"].iloc[-1])

# Define the filename for the Excel file
filename = (
    prefix
    + "_properties_postprocessed_"
    + "for_"
    + first_compound
    + "_to_"
    + last_compound
    + ".xlsx"
)

# export to excel
with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
    all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in all_df_master.columns:
        column_width = max(
            all_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = all_df_master.columns.get_loc(column)
        writer.sheets["All_Conformer_Properties"].set_column(
            col_idx, col_idx, column_width
        )
    properties_df_master.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master.columns:
        column_width = max(
            properties_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = properties_df_master.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)

+----+---------------+----------------------+------------------------+----------+----------+------------------------+------------+---------------+-----------------------+------------------------+----------+----------+-----------------------+------------+---------------+----------------------+------------------------+-----------+-----------+------------------------+-----------+------------+---------------------+------------------------+---------+---------+------------------------+---------+------------+----------------------+------------------------+---------+---------+-----------------------+---------+------------+------------------------+------------------------------+----------------------+----------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------------+------------------------+------------------------+--------------------------+--------------------------+-----------------------------+----

In [12]:
# Define filenames for the pickle files
pkl_filename_all = (
    prefix
    + "_properties_postprocessed"
    + "_all_conformer_properties"
    + "_for_"
    + prefix
    + ".pkl"
)
pkl_filename_summary = (
    prefix
    + "_properties_postprocessed"
    + "_summary_properties"
    + "_for_"
    + prefix
    + ".pkl"
)

# Save to pickle
all_df_master.to_pickle(pkl_filename_all)
properties_df_master.to_pickle(pkl_filename_summary)